In [10]:
# Imports
import sqlite3

from graphviz import Graph

from debugprov.validity import Validity
from debugprov.node import Node
from debugprov.execution_tree import ExecutionTree
from debugprov.execution_tree_creator import ExecTreeCreator
from debugprov.top_down import TopDown
from debugprov.heaviest_first import HeaviestFirst
from debugprov.visualization import Visualization
from debugprov.provenance_enhancement import ProvenanceEnhancement 
from debugprov.single_stepping import SingleStepping
from debugprov.divide_and_query import DivideAndQuery

In [11]:
NOW2_SQLITE_PATH = 'C:/Users/linha/Desktop/ws/py-scripts-examples/statistics/.noworkflow/db.sqlite'

In [12]:
CURSOR = sqlite3.connect(NOW2_SQLITE_PATH).cursor()

In [13]:
creator = ExecTreeCreator(CURSOR)
exec_tree = creator.create_exec_tree()

In [14]:
prov = ProvenanceEnhancement(exec_tree, CURSOR)

In [15]:
len(prov.dependencies)

301

In [16]:
class CustomVisualization(Visualization):
    
    # BEGIN CUSTOM ATTRIBUTES
    NODE_FUNC_CALL_COLOR = 'beige'
    NODE_NOT_FUNC_CALL_COLOR = 'grey60'
    # END CUSTOM ATTRIBUTES
    
    def generate_exec_tree(self, graph_name = 'exec_tree'):
        file_name = "{}.gv".format(graph_name)
        self.graph = Graph(graph_name, filename=file_name)
        self.graph.attr('node', shape='box')
        self.graph.attr('graph', ordering='out')
        root_node = self.exec_tree.root_node
        #self.graph.node(str(root_node.ev_id), root_node.get_name(), fillcolor=self.INVALID_COLOR, style='filled') # root node
        self.graph.node(str(root_node.ev_id), root_node.get_name())
        self.navigate(root_node)
        eval_node = self.exec_tree.node_under_evaluation
        if eval_node is not None:
            self.graph.node(str(eval_node.ev_id), str(eval_node.get_name()), fillcolor=self.NODE_IN_EVALUATION_COLOR, style='filled')
        buggy_node = self.exec_tree.buggy_node
        if buggy_node is not None:
            self.graph.node(str(buggy_node.ev_id), str(buggy_node.get_name()), fillcolor=self.BUGGY_NODE_COLOR, style='filled')
        if self.exec_tree.dependencies is not None:
            for d in self.exec_tree.dependencies: # this loop draws the provenance links between nodes
                ## BEGIN customization
                if (d.dependent.code_component_type == 'call'):
                    self.graph.node(str(d.dependent.ev_id), "{} {}".format(d.dependent.ev_id,d.dependent.code_component_name), fillcolor=self.NODE_FUNC_CALL_COLOR, style='filled')
                else:
                    self.graph.node(str(d.dependent.ev_id), "{} {}".format(d.dependent.ev_id,d.dependent.code_component_name), fillcolor=self.NODE_NOT_FUNC_CALL_COLOR, style='filled')
   
                if (d.influencer.code_component_type == 'call'):
                    self.graph.node(str(d.influencer.ev_id), "{} {}".format(d.influencer.ev_id,d.influencer.code_component_name), fillcolor=self.NODE_FUNC_CALL_COLOR, style='filled')
                else:
                    self.graph.node(str(d.influencer.ev_id), "{} {}".format(d.influencer.ev_id,d.influencer.code_component_name), fillcolor=self.NODE_NOT_FUNC_CALL_COLOR, style='filled')
                ## END customization
                self.graph.edge(str(d.dependent.ev_id), str(d.influencer.ev_id), None, color=self.PROVENANCE_EDGE_COLOR, dir='forward')

    def navigate(self, node:Node):
        chds = node.childrens
        for n in chds:
            #self.graph.edge(str(node.ev_id), str(n.ev_id), None, dir='forward')
            if n.validity == Validity.INVALID:
                self.graph.node(str(n.ev_id), str(n.get_name()), fillcolor=self.INVALID_COLOR, style='filled')
            elif n.validity == Validity.VALID: 
                self.graph.node(str(n.ev_id), str(n.get_name()), fillcolor=self.VALID_COLOR, style='filled')
            elif n.validity == Validity.UNKNOWN:  
                self.graph.node(str(n.ev_id), str(n.get_name()))
            elif n.validity is Validity.NOT_IN_PROV:
                self.graph.node(str(n.ev_id), str(n.get_name()), fillcolor=self.PROV_PRUNED_NODE_COLOR, style='filled')
        for n in chds: 
            self.navigate(n)  

In [17]:
vis = CustomVisualization(prov.exec_tree)

In [19]:
vis.view_exec_tree('exec_tree_prov_full')